### FedAvg algorithm, currently working for both Logistic Regression and 1 hidden layer NN.
##### See bottom for specific calls
https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be


In [1]:
%run -i DataCorruption.ipynb

In [93]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F

## MODELS

In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self, model_params):
        super(NeuralNetwork, self).__init__()
        
        input_dim = model_params["input_dim"]
        hidden_dim = model_params["hidden_dim"]
        output_dim = model_params["output_dim"]
        
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        return out

In [125]:
class Net_MNIST(nn.Module):
    def __init__(self, model_params):
        super(Net_MNIST, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 32 * 4 * 4)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        
        
class Net_CIFAR10(nn.Module):
    def __init__(self, model_params):
        super(Net_CIFAR10, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
            x = self.pool1(torch.relu(self.conv1(x)))
            x = self.pool2(torch.relu(self.conv2(x)))
            x = x.view(-1, 32 * 5 * 5)
            x = torch.relu(self.fc1(x))
            x = torch.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        


In [ ]:
class LogisticRegression(nn.Module):
    def __init__(self, model_params):
        super(LogisticRegression, self).__init__()
        
        input_dim = model_params["input_dim"]
        output_dim = model_params["output_dim"]
        
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out


## fedAvg algorithm

In [113]:

# Currently only allows for scrambling of labels as the corruption method

def FedAvg(model, data, model_params, training_params, corrupt=False, cp=0.5):
    model_dict = {1: NeuralNetwork, 2: LogisticRegression, 3: "Net"}
    model = model_dict[model]
    
    num_clients = training_params["num_clients"]
    epochs = training_params["epochs"]
    batch_size = training_params["batch_size"]
    lr = training_params["lr"]
    
    
    if data == "MNIST":
        # Load MNIST dataset - Normalized (MEAN STD)
        transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
        train_dataset = datasets.MNIST('../data', train=True, download=True, transform=transform)
        if model == "Net":
            model = Net_MNIST
    
    if data == "CIFAR10":
        transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, padding=4),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        train_dataset = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
        if model == "Net":
            model = Net_CIFAR10

    # Split data among clients
    client_datasets = torch.utils.data.random_split(train_dataset, [len(train_dataset) // num_clients] * num_clients)
    
    # Initialize global model
    central_model = model(model_params)

    # Train global model using federated averaging
    central_optimizer = optim.SGD(central_model.parameters(), lr=lr)
    central_criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        central_model.train()

        # Train local models on each client
        local_models = []
        for client_dataset in client_datasets:
            local_model = model(model_params)
            local_model.load_state_dict(central_model.state_dict())
            local_optimizer = optim.SGD(local_model.parameters(), lr=lr, momentum=0.9)
            local_criterion = nn.CrossEntropyLoss()

            for local_epoch in range(epochs):
                local_model.train()

                for local_data, local_target in torch.utils.data.DataLoader(client_dataset, batch_size=batch_size, shuffle=True):
                    if corrupt:
                        local_target = torch.as_tensor(data_corruption(3, local_data, local_target.tolist(), cp)).type(torch.LongTensor)
                    local_optimizer.zero_grad()
                    if model == Net_MNIST or model == Net_CIFAR10:
                        local_output = local_model(local_data)
                    else:
                        local_output = local_model(local_data.view(local_data.shape[0], -1))
                        
                    local_loss = local_criterion(local_output, local_target)
                    local_loss.backward()
                    local_optimizer.step()

            local_models.append(local_model)

        # Update central model using federated averaging
        for name, param in central_model.named_parameters():
            if name.endswith('.bias'):
                continue

            local_params = torch.stack([local_model.state_dict()[name] for local_model in local_models])
            central_mean = local_params.mean(0)
            param.data = central_mean.data

        central_loss = 0
        central_accuracy = 0
        central_optimizer.zero_grad()

        for central_data, central_target in torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True):
            if model == Net_MNIST or model == Net_CIFAR10:
                central_output = central_model(central_data)
            else:
                central_output = central_model(central_data.view(central_data.shape[0], -1))
            central_loss += central_criterion(central_output, central_target)
            central_accuracy += (central_output.argmax(1) == central_target).float().sum()

        central_loss /= len(train_dataset)
        central_accuracy /= len(train_dataset)

        central_loss.backward()
        central_optimizer.step()

        print(f'Epoch {epoch+1} - Global Training Loss: {central_loss:.4f}, Global Training Accuracy: {central_accuracy:.4f}')


## Tests

### NN on MNIST

In [119]:
training_params = {"epochs": 2, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"input_dim": 784, "hidden_dim": 128, "output_dim": 10}
FedAvg(1, "MNIST", model_params, training_params, corrupt=True, cp=0.9)

Epoch 1 - Global Training Loss: 0.0696, Global Training Accuracy: 0.3762
Epoch 2 - Global Training Loss: 0.0696, Global Training Accuracy: 0.4639


### Logistic Regression on MNIST

In [120]:
training_params = {"epochs": 2, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"input_dim": 728, "output_dim": 10}
FedAvg(2, "MNIST", model_params, training_params, corrupt=True, cp=0.9)

Epoch 1 - Global Loss: 0.0694, Global Accuracy: 0.2085
Epoch 2 - Global Loss: 0.0683, Global Accuracy: 0.2913


### CNN on MNIST

In [134]:
training_params = {"epochs": 2, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"in_channels": 1}
FedAvg(3, "MNIST", model_params, training_params, corrupt=True, cp=0.9)

Epoch 1 - Global Training Loss: 0.0718, Global Training Accuracy: 0.1016
Epoch 2 - Global Training Loss: 0.0717, Global Training Accuracy: 0.1827


### Logistic regression on CIFAR10

In [41]:
training_params = {"epochs": 2, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"input_dim": 3072, "output_dim": 10}
FedAvg(2, "CIFAR10", model_params, training_params, corrupt=True, cp=0.9)

Files already downloaded and verified
Epoch 1 - Global Training Loss: 0.0729, Global Training Accuracy: 0.0916
Epoch 2 - Global Training Loss: 0.0717, Global Training Accuracy: 0.1251


### NN on CIFAR10

In [43]:
training_params = {"epochs": 2, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"input_dim": 3072, "hidden_dim": 256, "output_dim": 10}
FedAvg(1, "CIFAR10", model_params, training_params, corrupt=True, cp=0.9)

Files already downloaded and verified
Epoch 1 - Global Training Loss: 0.0716, Global Training Accuracy: 0.1374
Epoch 2 - Global Training Loss: 0.0716, Global Training Accuracy: 0.1451


In [132]:
training_params = {"epochs": 8, "lr": 0.01, "batch_size": 32, "num_clients": 4}
model_params = {"input_dim": 3072, "hidden_dim": 256, "output_dim": 10}
FedAvg(1, "CIFAR10", model_params, training_params, corrupt=False, cp=0.9)

Files already downloaded and verified
Epoch 1 - Global Training Loss: 0.0527, Global Training Accuracy: 0.4062
Epoch 2 - Global Training Loss: 0.0496, Global Training Accuracy: 0.4424
Epoch 3 - Global Training Loss: 0.0476, Global Training Accuracy: 0.4638
Epoch 4 - Global Training Loss: 0.0464, Global Training Accuracy: 0.4793
Epoch 5 - Global Training Loss: 0.0453, Global Training Accuracy: 0.4917
Epoch 6 - Global Training Loss: 0.0444, Global Training Accuracy: 0.4991
Epoch 7 - Global Training Loss: 0.0439, Global Training Accuracy: 0.5055
Epoch 8 - Global Training Loss: 0.0435, Global Training Accuracy: 0.5061


### CNN on CIFAR10

In [126]:
training_params = {"epochs": 10, "lr": 0.01, "batch_size": 64, "num_clients": 4}
model_params = {}
FedAvg(3, "CIFAR10", model_params, training_params, corrupt=False, cp=0.9)

Files already downloaded and verified
Epoch 1 - Global Training Loss: 0.0300, Global Training Accuracy: 0.2935
Epoch 2 - Global Training Loss: 0.0337, Global Training Accuracy: 0.2567
Epoch 3 - Global Training Loss: 0.0330, Global Training Accuracy: 0.2974
Epoch 4 - Global Training Loss: 0.0310, Global Training Accuracy: 0.3210
Epoch 5 - Global Training Loss: 0.0276, Global Training Accuracy: 0.3964
Epoch 6 - Global Training Loss: 0.0247, Global Training Accuracy: 0.4512
Epoch 7 - Global Training Loss: 0.0247, Global Training Accuracy: 0.4620
Epoch 8 - Global Training Loss: 0.0255, Global Training Accuracy: 0.4565
Epoch 9 - Global Training Loss: 0.0226, Global Training Accuracy: 0.5046
Epoch 10 - Global Training Loss: 0.0215, Global Training Accuracy: 0.5321


#### Simple training loop to test the CNN w/o federated learning

In [130]:
from torch.utils.data import DataLoader

def train(model, trainloader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
        epoch_loss = running_loss / len(trainloader)
        epoch_acc = 100 * correct / total
        print('[Epoch %d] loss: %.3f | accuracy: %.3f' % (epoch + 1, epoch_loss, epoch_acc))
    
    print('Finished Training')
model_params = {}
# Load the CIFAR10 dataset
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

# Define the model, loss function, and optimizer
net = Net_CIFAR10(model_params)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Train the model
train(net, trainloader, criterion, optimizer, epochs=2)

Files already downloaded and verified
[Epoch 1] loss: 2.093 | accuracy: 21.180
[Epoch 2] loss: 2.192 | accuracy: 16.078
Finished Training


### Random Forest - Scikit-Learn

In [133]:
from sklearn.ensemble import RandomForestClassifier

# Load the MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=False)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=False)

# Extract the features and labels from the dataset
X_train = train_dataset.data.view(len(train_dataset), -1).numpy()
y_train = train_dataset.targets.numpy()
X_test = test_dataset.data.view(len(test_dataset), -1).numpy()
y_test = test_dataset.targets.numpy()

# Initialize the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier
rf.fit(X_train, y_train)

# Evaluate the classifier on the test set
accuracy = rf.score(X_test, y_test)

print(f"Test accuracy: {accuracy}")

Test accuracy: 0.9705


# HuggingFace

In [2]:
from datasets import load_dataset

food = load_dataset("food101", split="train[:5000]")

C:\Users\Nedan\anaconda3\envs\ML_project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset food101 downloaded and prepared to C:/Users/Nedan/.cache/huggingface/datasets/food101/default/0.0.0/7cebe41a80fb2da3f08fcbef769c8874073a86346f7fb96dc0847d4dfc318295. Subsequent calls will reuse this data.


In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
food = food.train_test_split(test_size=0.2)

In [7]:
labels = food["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [9]:
id2label[str(79)]

'prime_rib'

In [10]:
from transformers import AutoImageProcessor

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

C:\Users\Nedan\anaconda3\envs\ML_project\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nedan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), ToTensor(), normalize])

In [12]:
def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

In [13]:
food = food.with_transform(transforms)

In [14]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [15]:
import evaluate

accuracy = evaluate.load("accuracy")

In [16]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [17]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    output_dir="my_awesome_food_model",
    remove_unused_columns=False,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=food["train"],
    eval_dataset=food["test"],
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
)

trainer.train()

OSError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.